# <a id='toc1_'></a>[Modélisation des mobilités quotidiennes des individus](#toc0_)

Le but de ce notebook est d'explorer une méthodologie pour prédire les séquences de déplacements d'un individu à partir de ses caratéristiques socio-économiques.  
Ces premiers travaux ont pour objectifs de qualifier la pertinence de l'utilisation des algorithmes du traitement du langage naturel pour cette tâche.  


Définitions : 
- **Déplacement** : un déplacement est défini par un triplet (mode de déplacement, motif de déplacement, distance parcourue)
- **Séquence de déplacements** : c'est une suite ordonnée de déplacement effectué par un individu sur une journée.

Cette métohodologie se divise en 2 parties :
1. Apprentissage d'un modèle d'embedding pour effectuer un plongement sémentique des couples (mode, motif)
2. Apprentissage d'un RNN "vector to sequence" pour la prédiction d'une séquence de déplacement à partir d'un vecteur de caratistique socio-économique 

**Sommaire**<a id='toc0_'></a>    
- [Modélisation des mobilités quotidiennes des individus](#toc1_)    
  - [Création dataset](#toc1_1_)    
    - [Récupération des données](#toc1_1_1_)    
      - [Features](#toc1_1_1_1_)    
      - [Trajectoires](#toc1_1_1_2_)    
    - [Préprocessing des données](#toc1_1_2_)    
  - [Embedding des couples (mode, motif)](#toc1_2_)    
    - [Description du modèle](#toc1_2_1_)    
    - [Entrainement du modèle](#toc1_2_2_)    
    - [Data viz des 2 premières dimensions de l'emmbedings](#toc1_2_3_)    
  - [RNN vector to Sequence](#toc1_3_)    
    - [Description du modèle](#toc1_3_1_)    
    - [Entrainement du modèle](#toc1_3_2_)    
    - [Exemple de prédiction](#toc1_3_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import pandas as pd
import plotly.express as px
from core import DataGetter, Mob2Vec, Features2Trajectory

## <a id='toc1_1_'></a>[Création dataset](#toc0_)

### <a id='toc1_1_1_'></a>[Récupération des données](#toc0_)
Les données utilisées sont issues de l'Enquète Mobilité des Personnes 2019 (EMP). Elles sont récupérées via le package mobility.

In [2]:
data_getter = DataGetter(n_individus=10000, group_modalities=True)

#### <a id='toc1_1_1_1_'></a>[Features](#toc0_)
One hot enconding des features CSP, type de commune et nombre de voitures.

In [3]:
data_getter.get_features()
data_getter.features.head()

,city_category_B,city_category_C,city_category_I,city_category_R,csp_1,csp_2,csp_3,csp_4,csp_5,csp_6,csp_7,csp_8,csp_no_csp,n_cars_0,n_cars_1,n_cars_2+
individual_id,,,,,,,,,,,,,,,,
110000011500001,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
110000011600002,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0
110000012400001,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
110000012500002,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
110000013500002,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


#### <a id='toc1_1_1_2_'></a>[Trajectoires](#toc0_)
Pour chaque individu nous récupérons une suite de séquence défini par un triplet (mode, motif, distance)

In [4]:
data_getter.get_trajectories()
data_getter.trajectories.head()

individual_id
110000011500001                    [[3, 1, 28.0], [3, 1, 28.0]]
110000011600002      [[3, 3, 30.0], [3, 2, 18.0], [3, 1, 12.0]]
110000012400001                                  [[5, 9, 50.0]]
110000012500002                    [[4, 3, 14.0], [4, 1, 14.0]]
110000013500002    [[3, 9, 5.3214103219], [3, 1, 5.3214103219]]
dtype: object

### <a id='toc1_1_2_'></a>[Préprocessing des données](#toc0_)

In [5]:
data_getter.pad_trajectories()
data_getter.distance_standardization()

## <a id='toc1_2_'></a>[Embedding des couples (mode, motif)](#toc0_)

L'objectif de ce modèle est d'apprendre un plongement sémentique de notre vocabulaire de déplacements.

Nous apprenons un modèle équivalent à word2vec : "un déplacement, définit par le couple (motif, mode), est à une trajectoire, ce qu'un mot est à une phrase"

### <a id='toc1_2_1_'></a>[Description du modèle](#toc0_)

In [6]:
mob2vec = Mob2Vec(data_getter, window_size=2, embedding_dim=2)
mob2vec.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 2)              164       
                                                                 
 global_average_pooling1d (G  (None, 2)                0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 82)                246       
                                                                 
Total params: 410
Trainable params: 410
Non-trainable params: 0
_________________________________________________________________


### <a id='toc1_2_2_'></a>[Entrainement du modèle](#toc0_)

In [7]:
mob2vec.fit(test_size=0.1, batch_size=500, epochs=30)

Epoch 1/50
97/97 [==============================] - 1s 3ms/step - loss: 4.3150 - val_loss: 4.2097
Epoch 2/50
97/97 [==============================] - 0s 2ms/step - loss: 4.0734 - val_loss: 3.9058
Epoch 3/50
97/97 [==============================] - 0s 2ms/step - loss: 3.7245 - val_loss: 3.5151
Epoch 4/50
97/97 [==============================] - 0s 2ms/step - loss: 3.3315 - val_loss: 3.1229
Epoch 5/50
97/97 [==============================] - 0s 2ms/step - loss: 2.9717 - val_loss: 2.7923
Epoch 6/50
97/97 [==============================] - 0s 2ms/step - loss: 2.6910 - val_loss: 2.5516
Epoch 7/50
97/97 [==============================] - 0s 2ms/step - loss: 2.5031 - val_loss: 2.4042
Epoch 8/50
97/97 [==============================] - 0s 2ms/step - loss: 2.3967 - val_loss: 2.3248
Epoch 9/50
97/97 [==============================] - 0s 2ms/step - loss: 2.3397 - val_loss: 2.2806
Epoch 10/50
97/97 [==============================] - 0s 2ms/step - loss: 2.3061 - val_loss: 2.2531
Epoch 11/50
97/97 [

In [8]:
px.line(
    pd.DataFrame(mob2vec.history.history),
    template="plotly_white",
    labels={"index": "epochs", "value": "loss"},
)

### <a id='toc1_2_3_'></a>[Data viz des 2 premières dimensions de l'emmbedings](#toc0_)

In [9]:
df_embedding = pd.DataFrame(columns=["Mode", "Motif", "x", "y"])
df_embedding[["x", "y"]] = mob2vec.embedding_layer.embeddings[:, :2]
df_embedding[["Mode", "Motif"]] = list(data_getter.vocab.keys())

px.scatter(df_embedding, x="x", y="y", color="Motif", symbol="Mode", template="plotly_white",
    labels={"x": "dim 1", "y": "dim 2"},)

## <a id='toc1_3_'></a>[RNN vector to Sequence](#toc0_)

### <a id='toc1_3_1_'></a>[Description du modèle](#toc0_)

Instanciation d'un RNN "vector to sequence".

- **Input** : données scoio économique d'un individu (csp, nb de voitures, type de commune) à l'instant t=0, et 0 à l'instant t>0. L'input est de dimension (n_individus, trajectory_len, n_features). Les features ont été "one-hot encoded".
- **Output** : séquence de déplacements sur une journée. Un déplacement est défini par le triplet (motif, mode, distance). L'output est de dimension (n_individus, trajectory_len, embedding_dim+1). 

In [10]:
feats2traj = Features2Trajectory(data_getter, mob2vec)
feats2traj.get_trajectories_for_rnn()
feats2traj.get_feature_for_rnn()

In [11]:
feats2traj.model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 18, 10)            1080      
                                                                 
 dense_1 (Dense)             (None, 18, 3)             33        
                                                                 
Total params: 1,113
Trainable params: 1,113
Non-trainable params: 0
_________________________________________________________________


### <a id='toc1_3_2_'></a>[Entrainement du modèle](#toc0_)

In [12]:
feats2traj.fit(test_size=0.33, batch_size=100, epochs=10)

Epoch 1/10
67/67 [==============================] - 1s 8ms/step - loss: 1.6090 - val_loss: 1.1735
Epoch 2/10
67/67 [==============================] - 0s 7ms/step - loss: 0.8431 - val_loss: 0.6429
Epoch 3/10
67/67 [==============================] - 0s 5ms/step - loss: 0.6203 - val_loss: 0.5521
Epoch 4/10
67/67 [==============================] - 0s 5ms/step - loss: 0.5677 - val_loss: 0.5154
Epoch 5/10
67/67 [==============================] - 0s 5ms/step - loss: 0.5395 - val_loss: 0.4913
Epoch 6/10
67/67 [==============================] - 0s 5ms/step - loss: 0.5193 - val_loss: 0.4742
Epoch 7/10
67/67 [==============================] - 0s 6ms/step - loss: 0.5027 - val_loss: 0.4582
Epoch 8/10
67/67 [==============================] - 0s 5ms/step - loss: 0.4908 - val_loss: 0.4485
Epoch 9/10
67/67 [==============================] - 0s 6ms/step - loss: 0.4824 - val_loss: 0.4414
Epoch 10/10
67/67 [==============================] - 0s 6ms/step - loss: 0.4769 - val_loss: 0.4373


In [13]:
px.line(
    pd.DataFrame(feats2traj.history.history),
    template="plotly_white",
    labels={"index": "epochs", "value": "loss"},
)

### <a id='toc1_3_3_'></a>[Exemple de prédiction](#toc0_)

In [14]:
trajectory_pred_lib, trajectory_true_lib = feats2traj.compare_pred_true(indiv=0)

1/1 [==============================] - 0s 169ms/step


In [15]:
print("Vérité :")
trajectory_true_lib

Vérité :


[('3', '3', 8.0000000001),
 ('3', '2', 7.8000000008),
 ('3', '2', 8.0000000001),
 ('3', '1', 2.0),
 ('3', '1', 7.8000000008),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0),
 ('<pad>', '<pad>', 0.0)]

In [16]:
print("Prédiction :")
trajectory_pred_lib

Prédiction :


[('3', '8', 11.016173),
 ('3', '8', 9.300759),
 ('3', '1', 6.362087),
 ('<pad>', '<pad>', 3.7639718),
 ('<pad>', '<pad>', 1.8977649),
 ('<pad>', '<pad>', 0.8792459),
 ('<pad>', '<pad>', 0.44650474),
 ('<pad>', '<pad>', 0.29411134),
 ('<pad>', '<pad>', 0.24714239),
 ('<pad>', '<pad>', 0.22717844),
 ('<pad>', '<pad>', 0.20622703),
 ('<pad>', '<pad>', 0.17980559),
 ('<pad>', '<pad>', 0.15198486),
 ('<pad>', '<pad>', 0.12776168),
 ('<pad>', '<pad>', 0.11014915),
 ('<pad>', '<pad>', 0.09988136),
 ('<pad>', '<pad>', 0.0961483),
 ('<pad>', '<pad>', 0.097441204)]